Name : Selvina Swarna

Moodle Id : 20102194

Division : C  

Subject : DL




<center>Experiment No : 07</center>



In [ ]:
import os

# Specify the directory path
directory = 'preview'

# Create the directory if it doesn't exist
if not os.path.exists(directory):
    os.makedirs(directory)


In [ ]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

img = load_img('/content/drive/MyDrive/download.jpeg')  # this is a PIL image
x = img_to_array(img)  # this is a Numpy array with shape (3, 150, 150)
x = x.reshape((1,) + x.shape)  # this is a Numpy array with shape (1, 3, 150, 150)

# the .flow() command below generates batches of randomly transformed images
# and saves the results to the `preview/` directory
i = 0
for batch in datagen.flow(x, batch_size=1,
                          save_to_dir='preview', save_prefix='download', save_format='jpeg'):
    i += 1
    if i > 20:
        break  # otherwise the generator would loop indefinitely

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

print(tf.__version__)

2.15.0


In [ ]:
fashion_mnist=keras.datasets.fashion_mnist

In [ ]:
from keras.datasets import fashion_mnist

(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

4422102/4422102 [==============================] - 0s 0us/step


In [ ]:
train_images=train_images/255.0
test_images=test_images/255.0

train_images[0].shape

(28, 28)

In [ ]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [ ]:
!pip install keras-tuner

In [ ]:
import keras
from keras.layers import Conv2D, Flatten, Dense
from kerastuner.tuners import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

def build_model(hp):
    model = keras.Sequential([
        Conv2D(
            filters=hp.Int('conv_1_filter', min_value=32, max_value=128),
            kernel_size=hp.Choice('conv_1_kernel', values=[3, 5]),
            activation='relu',
            input_shape=(28, 28, 1)
        ),
        Conv2D(
            filters=hp.Int('conv_2_filter', min_value=32, max_value=64),
            kernel_size=hp.Choice('conv_2_kernel', values=[3, 5]),
            activation='relu'
        ),
        Flatten(),
        Dense(
            units=hp.Int('dense_1_units', min_value=32, max_value=128),
            activation='relu'
        ),
        Dense(10, activation='softmax')
    ])

    model.compile(
        optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

tuner_search = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    directory='output',
    project_name='fashion_mnist'
)

# Assuming you have already loaded the data
tuner_search.search(train_images, train_labels, epochs=3, validation_split=0.1)

Trial 5 Complete [00h 00m 26s]
val_accuracy: 0.8841666579246521

Best val_accuracy So Far: 0.9056666493415833
Total elapsed time: 00h 03m 05s


In [ ]:
model=tuner_search.get_best_models(num_models=1)[0]

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 80)        800       
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 32)        23072     
                                                                 
 flatten (Flatten)           (None, 18432)             0         
                                                                 
 dense (Dense)               (None, 68)                1253444   
                                                                 
 dense_1 (Dense)             (None, 10)                690       
                                                                 
Total params: 1278006 (4.88 MB)
Trainable params: 1278006 (4.88 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
history = model.fit(train_images, train_labels, epochs=10, validation_split=0.1)

Epoch 1/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.1483 - accuracy: 0.9444 - val_loss: 0.2504 - val_accuracy: 0.9165
Epoch 2/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.1075 - accuracy: 0.9598 - val_loss: 0.2742 - val_accuracy: 0.9170
Epoch 3/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0759 - accuracy: 0.9716 - val_loss: 0.3424 - val_accuracy: 0.9163
Epoch 4/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0538 - accuracy: 0.9805 - val_loss: 0.3481 - val_accuracy: 0.9145
Epoch 5/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0412 - accuracy: 0.9850 - val_loss: 0.4143 - val_accuracy: 0.9147
Epoch 6/10
1688/1688 [==============================] - 8s 4ms/step - loss: 0.0314 - accuracy: 0.9886 - val_loss: 0.4524 - val_accuracy: 0.9063
Epoch 7/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.0267 - accuracy: 0.9902 - val_loss: 0.5168 - val_accuracy: